# 0️⃣ 기본 전처리 (Rank 붙이기)

In [1]:
import pandas as pd

def load_with_rank(path):
    df = pd.read_csv(path)
    df["rank"] = df.groupby("user").cumcount() + 1
    return df


In [ ]:

EASE_df = load_with_rank("/data/ephemeral/home/minyou/data/submissions/EASE_best.csv")
Bert4Rec_df = load_with_rank("/data/ephemeral/home/minyou/data/submissions/bert4rec_predictions_upgrade.csv")
multiVAE_df = load_with_rank("/data/ephemeral/home/minyou/data/submissions/multivae_leave_one_out.csv")



## movie_df 

# 1️⃣ 모델 간 추천 겹침 분석 (Rank 고려)

In [16]:
def mean_overlap(df_a, df_b, k=10):
    users = set(df_a.user) & set(df_b.user)
    overlaps = []

    for u in users:
        a = set(df_a[(df_a.user == u) & (df_a["rank"] <= k)].item)
        b = set(df_b[(df_b.user == u) & (df_b["rank"] <= k)].item)
        overlaps.append(len(a & b))

    return sum(overlaps) / len(overlaps)


In [17]:

j_ease_bert = mean_overlap(EASE_df, Bert4Rec_df)
j_ease_vae  = mean_overlap(EASE_df, multiVAE_df)
j_bert_vae  = mean_overlap(Bert4Rec_df, multiVAE_df)


In [18]:
j_ease_bert

0.8552614795918367

In [19]:
j_ease_vae

4.58890306122449

In [20]:
j_bert_vae

0.8982461734693877

## 1-1) 상위 유저 vs 하위 유저 분리 분석 (이거 핵심) 2-1. user activity 기준 상위, 하위 N%만 보기
- -> 가설 : user activity 가 상위인건 모델별 큰 차이 없을것 또는 하위의 경우 그냥 유명한것만 추천하는 경향? -> 유명한 아이템만 추천되어 -> (개인화 추천이 안이루어진다) 
->모델별로 별 차이 없을것?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def split_users_by_activity(train_ratings, top_pct=0.1, bottom_pct=0.5):
    # train_ratings: user,item,time 같은 원본 (activity 계산용)
    act = train_ratings.groupby("user").size().rename("activity")
    q_top = act.quantile(1 - top_pct)
    q_bot = act.quantile(bottom_pct)
    heavy_users = set(act[act >= q_top].index)
    light_users = set(act[act <= q_bot].index)
    return act, heavy_users, light_users

def per_user_overlap(df_a, df_b, k=10, users=None):
    # df_a/df_b: user,item,rank
    if users is None:
        users = set(df_a.user) & set(df_b.user)
    else:
        users = set(users) & set(df_a.user) & set(df_b.user)

    out = []
    for u in users:
        a = set(df_a[(df_a.user==u) & (df_a.rank<=k)].item)
        b = set(df_b[(df_b.user==u) & (df_b.rank<=k)].item)
        out.append(len(a & b))
    return pd.Series(out, name=f"overlap@{k}")

def compare_groups_overlap(df_a, df_b, heavy_users, light_users, k=10):
    heavy = per_user_overlap(df_a, df_b, k=k, users=heavy_users)
    light = per_user_overlap(df_a, df_b, k=k, users=light_users)
    return heavy, light

def plot_overlap_hist(series_dict, title):
    plt.figure()
    for label, s in series_dict.items():
        plt.hist(s, bins=np.arange(-0.5, 11.5, 1), alpha=0.5, label=f"{label} (mean={s.mean():.2f})")
    plt.xticks(range(0, 11))
    plt.title(title)
    plt.xlabel("Intersection count in Top-10")
    plt.ylabel("Users")
    plt.legend()
    plt.show()


## 1-2)“어떤 rank가 제일 비슷한가?”
- 같은 rank에서 같은 item이 나오는 비율 (정확 일치)

- - 예: A의 rank=1 아이템 == B의 rank=1 아이템 ?

In [ ]:
def rank_exact_match_rate(df_a, df_b, k=10, users=None):
    if users is None:
        users = set(df_a.user) & set(df_b.user)
    else:
        users = set(users) & set(df_a.user) & set(df_b.user)

    # (user, rank) -> item 맵
    a_map = df_a[df_a.rank<=k].set_index(["user","rank"])["item"]
    b_map = df_b[df_b.rank<=k].set_index(["user","rank"])["item"]

    rates = {}
    for r in range(1, k+1):
        idx = [(u,r) for u in users if (u,r) in a_map.index and (u,r) in b_map.index]
        if not idx:
            rates[r] = np.nan
            continue
        match = (a_map.loc[idx].values == b_map.loc[idx].values).mean()
        rates[r] = match
    return pd.Series(rates, name="exact_match_rate")


# 2️⃣ 모델별 Popularity Bias 분석 (중요)
👉 해석
- 상위 rank일수록 popularity 급상승 → 안전한 추천
- rank별로 완만 → 취향 기반 추천

In [ ]:
# item_popularity: item -> interaction_count
merged = df.merge(item_popularity, on="item")

merged.groupby("rank")["interaction_count"].mean()


In [ ]:
merged["interaction_count"].mean()
merged["interaction_count"].quantile([0.5, 0.9, 0.99])


# 3️⃣ Coverage (모델 성격 한 방에 드러남)
👉
- coverage 낮음 → 반복 추천
- 높음 → 탐색 성향

In [ ]:
def coverage(df):
    return df.item.nunique()


# 4️⃣ Heavy / Light User 분석 (너한테 특히 중요 ⭐)

비교:
- overlap
- popularity
- diversity

👉 거의 항상 결과 갈림
→ “이 모델은 heavy user 전용이다” 라는 결론 가능

In [ ]:
user_activity = train.groupby("user").size()

df = df.merge(user_activity.rename("activity"), on="user")

heavy = df[df.activity >= user_activity.quantile(0.9)]
light = df[df.activity <= user_activity.quantile(0.5)]


# 5️⃣ 모델별 “유니크 추천 아이템”
이 아이템들의:
- popularity
- genre
- year

👉 Model A는 Model B가 추천하지 않는 저빈도 영화들을 적극적으로 추천

In [ ]:
def unique_items(df_a, df_b):
    return set(df_a.item) - set(df_b.item)
